In order to run the usage example, we first have to load the modules in the project directory. [See for more details.](https://stackoverflow.com/questions/15514593/importerror-no-module-named-when-trying-to-run-python-script)

Execute the following commands below:

In [1]:
import os

project_parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))


In [ ]:
project_parent_dir

In [2]:
import sys 
sys.path.append(project_parent_dir)

This example simplifies many aspects of a real-world implementation, such as concurrency, event versioning, and snapshotting for performance. In practice, frameworks and libraries are often used to handle the complexity of event sourcing and aggregates, and the event store might be a database designed for event storage.

In [ ]:
from domain.event_store import EventStore
from event_sourcing_sample.bank_account_aggregate import BankAccount
from event_sourcing_sample.helpers import rebuild_aggregate


def sample_usage():
    """
    This example simplifies many aspects of a real-world implementation, such as concurrency, event versioning, and snapshotting for performance. In practice, frameworks and libraries are often used to handle the complexity of event sourcing and aggregates, and the event store might be a database designed for event storage.
    """
    event_store = EventStore()

    # Create a new bank account
    bank_account = BankAccount(id="123")
    bank_account.create_account("John Doe")
    bank_account.deposit_money(100)

    # Persist changes (events) to the event store
    event_store.save_events(bank_account.id, bank_account.changes, expected_version=0)

    # Print history of events for auditing
    events = event_store.get_events_for_aggregate("123")
    print(f"Account History: \n")
    for event in events:
        print(
            f"Event Type: {event.type}, Data: {event.data}, Timestamp: {event.timestamp}"
        )

    # Rebuild the bank account's state from events
    reconstructed_account = rebuild_aggregate(event_store, "123")
    print(f"Reconstructed Account Balance: {reconstructed_account.balance}")


if __name__ == "__main__":
    sample_usage()

In [3]:
from domain.event_store import EventStore
from event_sourcing_sample.bank_account_v2_aggregate import BankAccountV2
from event_sourcing_sample.helpers import rebuild_aggregate


def sample_usage_v2():
    """
    This example simplifies many aspects of a real-world implementation, such as concurrency, event versioning, and snapshotting for performance. In practice, frameworks and libraries are often used to handle the complexity of event sourcing and aggregates, and the event store might be a database designed for event storage.
    """
    # Initialize the EventStore
    event_store = EventStore()
    # Create a new bank account id
    account_id = "123"

    # Register a BankAccount aggregate type for a specific aggregate ID
    event_store.register_aggregate_type(account_id, BankAccountV2)
    # Create a new bank account
    bank_account = BankAccountV2(id=account_id)
    bank_account.create_account("John Doe")
    bank_account.deposit_money(100)

    # Persist changes (events) to the event store
    event_store.save_events(bank_account.get_id(), bank_account.get_uncommitted_changes(), expected_version=0)

    # Print history of events for auditing
    events = event_store.get_events_for_aggregate(account_id)
    print(f"Account History: \n")
    for event in events:
        print(
            f"Event Type: {event.type}, Data: {event.data}, Timestamp: {event.timestamp}"
        )

    # Rebuild the bank account's state from events
    reconstructed_account = rebuild_aggregate(event_store, account_id)
    print(f"Reconstructed Account Balance: {reconstructed_account.balance}")


if __name__ == "__main__":
    sample_usage_v2()

Account History: 

Event Type: AccountCreated, Data: {'account_id': '123', 'owner': 'John Doe'}, Timestamp: 2024-03-18 00:26:34.825739
Event Type: MoneyDeposited, Data: {'account_id': '123', 'amount': 100}, Timestamp: 2024-03-18 00:26:34.825739
Applying event: AccountCreated
Applying event: MoneyDeposited
Reconstructed Account Balance: 100
